## COMP3670/6670 Programming Assignment 4 - Gaussian mixture models ##
---

**Enter Your Student ID:**

**Your Name:**
    
**Deadline:** Monday 23 October 23:59pm

**Submit:** Write your answers in this file, and submit a single Jupyter Notebook file (.ipynb) on Wattle. Rename this file with your student number as 'uXXXXXXX.ipynb'. Note: you don't need to submit the .png or .npy files. 

**Enter Discussion Partner IDs Below:**
Please implement things by yourself. If you use any external resources, list them here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
np.random.seed(42)

### **Task 0: Warming Up** ###

We first load and visualise the data, initialise the Gaussian mixture components randomly and visualise them.

In [ ]:
# load data
x = np.loadtxt("./data/gmm_data.txt")
N, D = x.shape[0], x.shape[1]

print("number of data points %d" % N)
print("number of dimensions %d" % D)

plt.figure()
plt.scatter(x[:, 0], x[:, 1], color='k', alpha=0.6)
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')

In [ ]:
def viz_mixture(X, mus, Sigmas, suffix=''):
    # code to visualise the Gaussian components
    # X is data, N x D. N: number of datapoints, D: dimensions
    # mus contains means, K x D, K: number of Gaussian components
    # Sigmas contains covariances, K x D x D.
    
    plt.figure()
    ax = plt.subplot(111)
    CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
    K = mus.shape[0]
    for k in range(K):
        m = mus[k, :]
        cov = Sigmas[k, :, :]
        lambda_, v = np.linalg.eig(cov)
        lambda_ = np.sqrt(lambda_)
        from matplotlib.patches import Ellipse
        for j in range(1, 4):
            ell = Ellipse(xy=m,
                        width=lambda_[0]*j*2, height=lambda_[1]*j*2,
                        angle=np.rad2deg(np.arccos(v[0, 0])))
            ell.set_facecolor('none')
            ell.set_edgecolor(CB_color_cycle[k])
            ax.add_artist(ell)
    plt.scatter(x[:, 0], x[:, 1], color='k', alpha=0.6)
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.xlim([-8, 8])
    plt.ylim([-8, 8])

In [ ]:
# initialise the mixture components randomly 
K = 2 # 2 components
init_pis = np.random.rand(K) 
init_pis = init_pis / np.sum(init_pis)
init_mus = np.random.uniform(-5, 5, K * D).reshape(K, D)
init_Sigmas = np.array([np.eye(D) for _ in range(K)])

# visualise
viz_mixture(x, init_mus, init_Sigmas)

### **Task 1: Computing GMM log-likelihood** ###

The GMM log-likelihood for N data points is:
$$
  \log p(x | \theta) = \sum_{n=1}^N \log \sum_{k=1}^K \pi_k \mathcal{N}(x_n; \mu_k, \Sigma_k)
$$

Write a function to compute the log-likelihood of the GMMs parameters given data. 

Hint: have a look at Week 9's tutorial and the `compute_responsibility` function in Task 2 below.


In [ ]:
def gmm_log_likelihood(x, mus, Sigmas, pis):
    # x is data, N x D. N: number of datapoints, D: dimensions
    # mus contains means, K x D, K: number of Gaussian components.
    # Sigmas contains covariances, K x D x D.
    # pis contains the mixture weights, of size K.
    # this function should return a number
    
    # your code here
    return 0

In [ ]:
# we can compute the log likelihood of the initial parameters
loglik = gmm_log_likelihood(x, init_mus, init_Sigmas, init_pis)
print(loglik)

### **Task 2: Estimate GMM parameters** ###

In Week 9's tutorial, you have implemented the E-step of the EM algorithm for GMMs to compure the assignment probability (see `compute_responsibility` below) . Your next task is to implement the updates required in the M-step.
$$
\mu_k = \frac{1}{N_k} \sum_{n=1}^{N} r_{nk} x_n, \quad N_k = \sum_{n=1}^N r_{nk}
$$
$$
\Sigma_k = \frac{1}{N_k} \sum_{n=1}^Nr_{nk}(x_n - \mu_k)(x_n-\mu_k)^\intercal
$$
$$
\pi_k = \frac{N_k}{N}
$$
Note that we are not asking you to handle the singularity issue in your code, just implement the updates above.

In [ ]:
def compute_responsibility(x, mus, Sigmas, pis):
    logpdfs = np.array([multivariate_normal.logpdf(x, mean=mus[k, :], cov=Sigmas[k, :, :]) for k in range(K)])
    logpi = np.log(pis)
    logpin = logpi[:, None] + logpdfs
    denom = logsumexp(logpin, axis=0)
    logres = logpin - denom[None, :]
    res = np.exp(logres).T
    return res
    
def update_mean(x, r):
    # x is data, N x D
    # r is responsibility, N x K
    
    # This function should return the new means, K x D
    # your code here

def update_cov(x, r, mus):
    # x is data, N x D
    # r is responsibility, N x K
    # mus is the current means, K x D
    
    # This function should return the new covariances, K x D x D
    # your code here

def update_pi(r):
    # r is responsibility, N x K
    # This function should return the new mixture weights, of length K
    # your code here
    

We now run a few EM iterations and and visualise the resulting mixture. Since we are not handling issues such as singularity, the following code block may fail. If that happens, try to reinitialise using the final code block in task 0.

In [ ]:
no_iters = 100
mus, Sigmas, pis = init_mus, init_Sigmas, init_pis
for i in range(no_iters):
    r = compute_responsibility(x, mus, Sigmas, pis)
    mus = update_mean(x, r)
    Sigmas = update_cov(x, r, mus)
    pis = update_pi(r)
viz_mixture(x, mus, Sigmas)

### **Task 3: Understanding the likelihood** ###

**Task 3.1**

In Task 1, you have computed the likelihood of the initial parameters. We can also compute the likelihood of the parameters given by EM. Compare the two likelihoods and comment.

In [ ]:
loglik = gmm_log_likelihood(x, mus, Sigmas, pis)
print(loglik)

*your answer goes here!*

**Task 3.2**

We can also evaluate the log likelihood given new test points. We will next visualise the log-likelihoods computed on training data and validation data when varying the number of mixture components. Comment on the result.

In [ ]:
x_valid = np.loadtxt("./data/gmm_data_valid.txt")
Ks = [1, 2, 4, 6, 8, 10, 12]
no_iters = 100
lik_train = []
lik_valid = []
for K in Ks:
    while True:
        try:
            init_pis = np.random.rand(K) 
            init_pis = init_pis / np.sum(init_pis)
            init_mus = np.random.uniform(-5, 5, K * D).reshape(K, D)
            init_Sigmas = np.array([np.eye(D) for _ in range(K)])
            mus, Sigmas, pis = init_mus, init_Sigmas, init_pis
            for i in range(no_iters):
                r = compute_responsibility(x, mus, Sigmas, pis)
                mus = update_mean(x, r)
                Sigmas = update_cov(x, r, mus)
                pis = update_pi(r)
            lik_train.append(gmm_log_likelihood(x, mus, Sigmas, pis))
            lik_valid.append(gmm_log_likelihood(x_valid, mus, Sigmas, pis))
            break
        except:
            # We just have another random restart
            continue
plt.figure()
plt.plot(Ks, lik_train, color='r', lw=3, label='train')
plt.plot(Ks, lik_valid, color='b', lw=3, label='valid')
plt.legend()
plt.xlabel('number of components, K')
plt.ylabel('log-likelihood')

*your comment goes here!*